In [1]:
import pandas as pd
import datetime
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from functools import reduce
from collections import Counter


In [2]:
df = pd.read_csv("../data/raw/train_8wry4cB.csv")

In [3]:
def standard_datetime(column,format: str ="%d/%m/%y %H:%M"):
    return pd.to_datetime(column , format=format)

In [4]:
df.startTime = standard_datetime(df.startTime)
df.endTime = standard_datetime(df.endTime)
df.rename(columns={"ProductList":"product_list","startTime":"start_time","endTime":"end_time"},inplace=True)
df.drop(columns=["session_id"],inplace=True)

# Making Datetime feature

In [5]:
def set_day(df):
    weeks_day = ['Monday', 'Friday', 'Saturday', 'Tuesday', 'Sunday', 'Wednesday', 'Thursday']
    a = df.start_time.dt.day_name()
    for i in weeks_day:
        df[i] = (a == i)
    return df

In [6]:
def set_month_start(df):
    df["month_start"] = df.start_time.dt.is_month_start
    return df

In [7]:
def set_weeks(df):
    weeks = [51, 50, 49, 47, 48, 46, 52]
    a = df.start_time.dt.weekofyear
    for i in weeks:
        df[f"week_{i}"] = (a==i)
    return df

In [8]:
def set_interval_time(df , interval="3H"):
    df["st"] = pd.to_datetime(df.start_time.dt.strftime("%H:%M"),format="%H:%M")
    bins = df.groupby(pd.Grouper(key = "st",freq=interval)).count().index
    bins = [(i,j-pd.Timedelta(seconds=1)) for i,j in zip(bins[:-1],bins[1:])] + [(bins[-1] , bins[0]- pd.Timedelta(seconds=1))]
    for i , j in bins:
        df[f"time_{i.time().hour}"] = df.st.between(i,j)
    return df

In [9]:
def set_duration(df):
    a =(df["end_time"] - df["start_time"]).dt.seconds/60
    a[a > a.quantile(0.97)] = a.quantile(0.97) 
    df["duration"] = a
    return df

In [10]:
set_day(df)
set_month_start(df)
set_weeks(df)
set_interval_time(df)
set_duration(df)
df.head()

,start_time,end_time,product_list,gender,Monday,Friday,Saturday,Tuesday,Sunday,Wednesday,...,st,time_0,time_3,time_6,time_9,time_12,time_15,time_18,time_21,duration
0,2014-12-15 18:11:00,2014-12-15 18:12:00,A00002/B00003/C00006/D28435/;A00002/B00003/C00...,female,True,False,False,False,False,False,...,1900-01-01 18:11:00,False,False,False,False,False,False,True,False,1.0
1,2014-12-16 14:35:00,2014-12-16 14:41:00,A00001/B00009/C00031/D29404/;A00001/B00009/C00...,male,False,False,False,True,False,False,...,1900-01-01 14:35:00,False,False,False,False,True,False,False,False,6.0
2,2014-12-01 15:58:00,2014-12-01 15:58:00,A00002/B00001/C00020/D16944/,female,True,False,False,False,False,False,...,1900-01-01 15:58:00,False,False,False,False,False,True,False,False,0.0
3,2014-11-23 02:57:00,2014-11-23 03:00:00,A00002/B00004/C00018/D10284/;A00002/B00004/C00...,female,False,False,False,False,True,False,...,1900-01-01 02:57:00,True,False,False,False,False,False,False,False,3.0
4,2014-12-17 16:44:00,2014-12-17 16:46:00,A00001/B00001/C00012/D30805/;A00001/B00001/C00...,male,False,False,False,False,False,True,...,1900-01-01 16:44:00,False,False,False,False,False,True,False,False,2.0


In [11]:
df.columns

Index(['start_time', 'end_time', 'product_list', 'gender', 'Monday', 'Friday',
       'Saturday', 'Tuesday', 'Sunday', 'Wednesday', 'Thursday', 'month_start',
       'week_51', 'week_50', 'week_49', 'week_47', 'week_48', 'week_46',
       'week_52', 'st', 'time_0', 'time_3', 'time_6', 'time_9', 'time_12',
       'time_15', 'time_18', 'time_21', 'duration'],
      dtype='object')

# Making Product List Feature


In [12]:
for index,value in enumerate(["cat" , "sub-cat","sub-sub-cat","product"]):
    df[value] = df.product_list.apply(lambda x : [i.split("/")[index] for i in x.split(";")])


In [13]:
def set_product_count(df):
    df["product_count"] = df["product"].apply(len)
    return df

In [14]:
def cat_sub_cat(df):
    # calculating male and female cat - sub-cat count 
    c_m = Counter()
    c_f = Counter()
    
    for index,rows in df.iterrows():
        a = [f"{i}-{j}"for i , j in zip(rows["cat"],rows["sub-cat"])]
        if rows["gender"] == "female":
            c_f.update(a)
        else:
            c_m.update(a)
    
    # cumsum for 97 percentile in female
    f = pd.DataFrame(c_f.items(),columns=["cat","value"]).sort_values(by="value",ascending=False).reset_index(drop=True)
    f["cu"] = (f.value.cumsum()/f.value.sum())
    f = f[f["cu"]<=0.975]
    
    # cumsum for 97 percentile in man
    m = pd.DataFrame(c_m.items(),columns=["cat","value"]).sort_values(by="value",ascending=False).reset_index(drop=True)
    m["cu"] = (f.value.cumsum()/f.value.sum())
    m = m[m["cu"]<=0.975]
    
    # merge to get "cat - sub-cat to be used for both of them"
    f = f.merge(m , how="outer",left_on="cat",right_on="cat", suffixes=('_female', '_male'))["cat"].values.tolist()
   
    m = df.apply(lambda x: [f"{i}-{j}" for i ,j in zip(x["cat"],x["sub-cat"])]     ,axis=1)
    m = m.apply(lambda x:  dict(Counter(x)))
    for i in f:df[i] = m.apply(lambda x: x.get(i,0))
    return df

In [15]:
def set_duration_product_ratio(df):
    df["duration_product_ratio"] = df["duration"]/df["product_count"]

In [16]:
def cat_flag(df):
    cat_list = reduce(lambda x,y :set(x).union(set(y)) , df.cat)
    for i in cat_list:
        df[i] = df.cat.apply(lambda x: i in x)
    return df

In [17]:
set_product_count(df)
cat_sub_cat(df)
set_duration_product_ratio(df)
cat_flag(df)
df.columns

Index(['start_time', 'end_time', 'product_list', 'gender', 'Monday', 'Friday',
       'Saturday', 'Tuesday', 'Sunday', 'Wednesday', 'Thursday', 'month_start',
       'week_51', 'week_50', 'week_49', 'week_47', 'week_48', 'week_46',
       'week_52', 'st', 'time_0', 'time_3', 'time_6', 'time_9', 'time_12',
       'time_15', 'time_18', 'time_21', 'duration', 'cat', 'sub-cat',
       'sub-sub-cat', 'product', 'product_count', 'A00002-B00002',
       'A00002-B00003', 'A00002-B00001', 'A00003-B00012', 'A00002-B00007',
       'A00002-B00004', 'A00003-B00022', 'A00002-B00005', 'A00002-B00016',
       'A00002-B00017', 'A00002-B00006', 'A00003-B00021', 'A00003-B00020',
       'A00001-B00001', 'A00003-B00004', 'A00001-B00009', 'A00003-B00028',
       'A00005-B00019', 'A00002-B00011', 'A00001-B00015', 'A00001-B00004',
       'A00003-B00026', 'A00005-B00032', 'A00004-B00014', 'A00011-B00050',
       'A00001-B00031', 'A00003-B00036', 'A00002-B00010', 'A00006-B00057',
       'A00003-B00039', 'A00005

# Splitting Train and Valid.

In [18]:
# drop these columns
df  = df.drop(columns=['start_time', 'end_time', 'product_list','st','cat', 'sub-cat','sub-sub-cat', 'product'])